# Prerequisites

## Load employees.csv into DataFame

In [0]:
df = spark.read.csv(path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/employee.csv",
        sep="|",
        inferSchema=True,
        header=True,
        quote="'",
    )
display(df)

In [0]:
df.select("id", "name").display()

In [0]:
from pyspark.sql.functions import col
df.select(col("id")).display()

In [0]:
from pyspark.sql.functions import col
df.select(col("id").alias("user_id")).display()

# Filter Operations

In [0]:
df.filter(col("desig")=="Developer").display()

In [0]:
df.filter((col("desig")=="Developer") | (col("desig")=="Team Lead")).display()

In [0]:
df.filter((col("desig")=="Developer") | (col("desig")=="Team Lead")).select(col("id")).display()

In [0]:
df.filter((col("desig")=="Developer") & (col("gen")=="M")).display()

In [0]:
df.filter(col("exp").between(1, 6)).display()

In [0]:
df.filter((col("exp").between(1, 6)) & (col("gen")=="M")).select(col("name")).display()

# Grouping, Aggregation and Sorting

In [0]:
df.groupBy(col("gen")).count().display()

In [0]:
# df.groupBy(col("gen")).count().sort(col("count"), ascending=False).display()
df.groupBy(col("gen")).count().sort(col("count").desc()).display()

In [0]:
from pyspark.sql.functions import avg, sum, min, max
df.groupBy(col("desig")).agg(min("exp").alias("min_exp"),
                        max("exp").alias("max_exp"),
                        avg("exp").alias("avg_exp")).display()

# Derive new columns

In [0]:
from pyspark.sql.functions import lit
df.withColumn("is_employed", lit("Yes")).display()

In [0]:
from pyspark.sql.functions import when

df.withColumn(
    "category",
    when(col("exp") >= 10, "Senior")
    .when(col("exp") >= 5, "Mid")
    .when(col("exp") >= 1, "Junior")
    .otherwise("Invalid Experience"),
).display()

# View

In [0]:
df.createOrReplaceTempView("employee_vw")

In [0]:
%sql
SELECT * FROM employee;

In [0]:
%sql
SELECT *,
CASE
  WHEN exp >= 10 THEN 'Senior'
  WHEN exp >= 5 THEN 'Mid Level'
  WHEN exp >= 1 THEN 'Junior'
  ELSE 'Invalid Experience'
END AS category
FROM employee_vw